# Grid setup LUT

## Imports

In [2]:
%load_ext autoreload
%autoreload 2

# Standard imports
from collections import defaultdict
import csv
import pickle 

# 3rd party imports
import numpy as np
np.set_printoptions(linewidth=1000)
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from pprint import pprint
from tqdm.notebook import tqdm

# Local imports
import modules.laplacian as laplacian
import modules.jacobian as jacobian

## Functions

In [3]:
def _create_correspondence(y: int, x: int, angle: int, magnitude: float, n: int):
    """
    Create a correspondence for the given parameters.
    
    Parameters:
    y (int): The y-coordinate of the pixel.
    x (int): The x-coordinate of the pixel.
    angle (int): The angle of the vector (degrees).
    magnitude (float): The magnitude of the vector.
    
    Returns:
    np.ndarray: The moving point.
    np.ndarray: The fixed point.
    """
    angle = np.radians(angle)
    mpoint = np.array([0, y, x])
    fpoint_y = y + np.sin(angle) * magnitude
    fpoint_x = x + np.cos(angle) * magnitude
    fpoint = np.array([0, fpoint_y, fpoint_x])
    # Check bounds
    if fpoint_y < 0 or fpoint_x < 0 or fpoint_y > n - 1 or fpoint_x > n - 1:
        raise Exception(f"Invalid correspondence, out of bounds: {mpoint} -> {fpoint}")
    return mpoint, fpoint


def create_case(y: int, x: int, angle: int, magnitude: float, base_mpoints: np.ndarray, base_fpoints: np.ndarray, n: int=7, pad=True):
    """
    Create a case for the given parameters.
    
    Parameters:
    y (int): The y-coordinate of the pixel.
    x (int): The x-coordinate of the pixel.
    angle (int): The angle of the vector (degrees).
    magnitude (float): The magnitude of the vector.
    base_mpoints (np.ndarray): The base moving points.
    base_fpoints (np.ndarray): The base fixed points.
    n (int): The size of the grid.
    
    Returns:
    np.ndarray: The deformation field.
    np.ndarray: The Jacobian determinant field.
    """
    #new_size = 15  # 15 x 15
    #pad_size = (new_size - n) // 2
    #fixed_resolution = np.zeros((1, new_size, new_size))
    fixed_resolution = np.zeros((1, n, n))
    # Create the correspondence and apply as the second correspondence in the grid
    mpoint, fpoint = _create_correspondence(y, x, angle, magnitude, n)
    #mpoint += np.array([0, pad_size, pad_size])
    #fpoint += np.array([0, pad_size, pad_size])
    curr_mpoints = base_mpoints.copy()
    curr_fpoints = base_fpoints.copy()
    curr_mpoints = np.append(curr_mpoints, [mpoint], axis=0)
    curr_fpoints = np.append(curr_fpoints, [fpoint], axis=0)
    # Run Laplacian and Jacobian determinant calculations and save the Jacobian determinant field
    deformation, _, _, _, _ = laplacian.sliceToSlice3DLaplacian(fixed_resolution, curr_mpoints, curr_fpoints)
    #deformation, _, _, _, _ = laplacian.sliceToSlice3DLaplacian(fixed_resolution, curr_fpoints, curr_mpoints)
    # Crop deformation field to original size
    #deformation = deformation[:, :, pad_size:pad_size + n, pad_size:pad_size + n]
    jdet_field = jacobian.sitk_jacobian_determinant(deformation)[0]
    return deformation, jdet_field


def show_data(data: dict, y: int, x: int, angle: int, magnitude: float, title="", fontsize=8):
    """
    Show the grid of deformation information.
    
    Parameters:
    data (dict): The data dictionary.
    y (int): The y-coordinate of the pixel.
    x (int): The x-coordinate of the pixel.
    angle (int): The angle of the vector.
    magnitude (float): The magnitude of the vector.
    deformation (np.ndarray): The deformation field.
    title (str): The title of the plot.
    fontsize (int): The fontsize of the text.
    """
    deformation, jdet_field = data[y][x][angle][magnitude]
    norm = mcolors.TwoSlopeNorm(vmin=min(jdet_field.min(), -1), vcenter=0, vmax=jdet_field.max())
    
    # Plot the grid of deformation information
    f = plt.figure(figsize=(10, 5))
    
    # Plot the grid of Jacobian determinants color values
    plt.imshow(jdet_field, cmap="seismic", norm=norm)
    plt.colorbar()
    angle_rad = np.radians(angle)
    plt.arrow(jdet_field.shape[1] // 2, jdet_field.shape[0] // 2, np.cos(0) * 1, np.sin(0) * 1, head_width=0.15, head_length=0.15, fc='green', ec='green')
    plt.arrow(x, y, np.cos(angle_rad) * magnitude, np.sin(angle_rad) * magnitude, head_width=0.15, head_length=0.15, fc='green', ec='green')
    
    # Draw a dot at moving points
    #for i in range(len(mpoints)):
    #    plt.scatter(mpoints[i][2], mpoints[i][1], color='green')
    
    # Plot the coordinate text
    for j in range(jdet_field.shape[0]):
        for i in range(jdet_field.shape[1]):
            # Get the pixel information
            normalized_y = j - (jdet_field.shape[0] // 2) 
            normalized_x = i - (jdet_field.shape[1] // 2)
            
            curr_displacement = deformation[:, 0, j, i]
            displacement_y = curr_displacement[1]
            displacement_x = curr_displacement[2]

            # Get the angle information
            curr_theta_rad = np.arctan2(displacement_y, displacement_x)
            if curr_theta_rad < 0:  # Ensure the angle is in the range [0, 2pi)
                curr_theta_rad += 2 * np.pi
            curr_theta = np.abs(np.degrees(curr_theta_rad))
            
            # Get the magnitude information
            curr_magnitude = np.linalg.norm(curr_displacement)
            jdet = jdet_field[j, i]
            
            # Setup text to display
            coord_text = f"({displacement_y:.1f}, {displacement_x:.1f})\n"
            jdet_text = f"{jdet:.2f} J"
            vector_text = f"\n{curr_theta:.2f}°\n" + f"∥{curr_magnitude:.2f}∥"
            #pixel_text = coord_text + jdet_text + vector_text
            pixel_text = jdet_text
            plt.text(i, j, pixel_text, color='black', ha='center', va='center', fontsize=fontsize, weight='normal')
            
            # Draw the vector direction
            magnitude_scale = 0.33  # To scale the vector magnitude for visualization purposes
            #plt.arrow(i, j, np.cos(curr_theta_rad) * magnitude_scale, np.sin(curr_theta_rad) * magnitude_scale, head_width=0.15, head_length=0.15, fc='black', ec='black')            

            

    # Set the xticks and yticks
    plt.xticks(np.arange(jdet_field.shape[0]), labels=np.arange(-(jdet_field.shape[1] // 2) - 1, jdet_field.shape[1] // 2) + 1)
    plt.yticks(np.arange(jdet_field.shape[0]), labels=np.arange(-(jdet_field.shape[0] // 2) - 1, jdet_field.shape[0] // 2) + 1)
    plt.title(title)
    plt.show()
    f.clear()
    plt.close(f)

### Flip case 2

In [11]:
def _create_correspondence2(y: int, x: int, angle: int, magnitude: float, n: int):
    """
    Create a correspondence for the given parameters.
    
    Parameters:
    y (int): The y-coordinate of the pixel.
    x (int): The x-coordinate of the pixel.
    angle (int): The angle of the vector (degrees).
    magnitude (float): The magnitude of the vector.
    
    Returns:
    np.ndarray: The moving point.
    np.ndarray: The fixed point.
    """
    angle = np.radians(angle)
    mpoint = np.array([0, y, x])
    fpoint_y = y + np.sin(angle) * magnitude
    fpoint_x = x + np.cos(angle) * magnitude
    fpoint = np.array([0, fpoint_y, fpoint_x])
    return mpoint, fpoint

def create_case2(y: int, x: int, angle: int, magnitude: float, base_mpoints: np.ndarray, base_fpoints: np.ndarray, n: int=7, pad=True, base_magnitude=1, x_1_case=False, downsample=False, multiplier=3, upscaled_offset=1):
    """
    Create a case for the given parameters.
    
    Parameters:
    y (int): The y-coordinate of the pixel.
    x (int): The x-coordinate of the pixel.
    angle (int): The angle of the vector (degrees).
    magnitude (float): The magnitude of the vector.
    base_mpoints (np.ndarray): The base moving points.
    base_fpoints (np.ndarray): The base fixed points.
    n (int): The size of the grid.
    
    Returns:
    np.ndarray: The deformation field.
    np.ndarray: The Jacobian determinant field.
    """
    #new_size = 15  # 15 x 15
    #pad_size = (new_size - n) // 2
    #fixed_resolution = np.zeros((1, new_size, new_size))
    fixed_resolution = np.zeros((1, n, n))
    # Create the correspondence and apply as the second correspondence in the grid
    mpoint, fpoint = _create_correspondence2(y, x, angle, magnitude, n)
    #mpoint += np.array([0, pad_size, pad_size])
    #fpoint += np.array([0, pad_size, pad_size])
    #base_fpoints = np.array([[0, n // 2, n // 2 + 1]])  # normal case
    #base_mpoints = np.array([[0, n // 2, n // 2]])
    
    #x_1_case = False
    if x_1_case:
        if base_fpoints is None:
            base_fpoints = np.array([[0, n // 2, n // 2 + base_magnitude - 1]])  # x - 1 case
        if base_mpoints is None:
            #base_mpoints = np.array([[0, n // 2, n // 2 - base_magnitude]])
            base_mpoints = np.array([[0, n // 2, n // 2 - 1]])
    else:  # else normal case
        if base_fpoints is None:
            base_fpoints = np.array([[0, n // 2, n // 2 + base_magnitude]])
        if base_mpoints is None:
            base_mpoints = np.array([[0, n // 2, n // 2]])
        
        
    curr_mpoints = base_mpoints.copy()
    curr_fpoints = base_fpoints.copy()
    curr_mpoints = np.append(curr_mpoints, [mpoint], axis=0)
    curr_fpoints = np.append(curr_fpoints, [fpoint], axis=0)
    # Run Laplacian and Jacobian determinant calculations and save the Jacobian determinant field
    deformation, _, _, _, _ = laplacian.sliceToSlice3DLaplacian(fixed_resolution, curr_mpoints, curr_fpoints)
    #deformation, _, _, _, _ = laplacian.sliceToSlice3DLaplacian(fixed_resolution, curr_fpoints, curr_mpoints)
    # Crop deformation field to original size
    #deformation = deformation[:, :, pad_size:pad_size + n, pad_size:pad_size + n]
    jdet_field = jacobian.sitk_jacobian_determinant(deformation)[0]
    
    if downsample:
        new_displacement_field = np.zeros((3, 1, 7, 7))
        for j in range(7):
            for i in range(7):
                curr_displacement = deformation[:, 0, (j * multiplier) + upscaled_offset, (i * multiplier) + upscaled_offset]
                displacement_y = curr_displacement[1] / multiplier
                displacement_x = curr_displacement[2] / multiplier
                new_displacement_field[0, 0, j, i] = 0
                new_displacement_field[1, 0, j, i] = displacement_y
                new_displacement_field[2, 0, j, i] = displacement_x
                    
        jdet_field = jacobian.sitk_jacobian_determinant(new_displacement_field)[0]
    
    return deformation, jdet_field


def show_data2(deformation, jdet_field, y: int, x: int, angle: int, magnitude: float, title="", fontsize=8, base_magnitude=1, figsize=(10, 5)):
    """
    Show the grid of deformation information.
    
    Parameters:
    data (dict): The data dictionary.
    y (int): The y-coordinate of the pixel.
    x (int): The x-coordinate of the pixel.
    angle (int): The angle of the vector.
    magnitude (float): The magnitude of the vector.
    deformation (np.ndarray): The deformation field.
    title (str): The title of the plot.
    fontsize (int): The fontsize of the text.
    """
    norm = mcolors.TwoSlopeNorm(vmin=min(jdet_field.min(), -1), vcenter=0, vmax=jdet_field.max())
    
    # Plot the grid of deformation information
    f = plt.figure(figsize=figsize)
    
    # Plot the grid of Jacobian determinants color values
    plt.imshow(jdet_field, cmap="seismic", norm=norm)
    plt.colorbar()
    angle_rad = np.radians(angle)
    
    # Draw arrow for center
    plt.arrow(jdet_field.shape[1] // 2, jdet_field.shape[0] // 2, np.cos(0) * base_magnitude, np.sin(0) * base_magnitude, head_width=0.15, head_length=0.15, fc='green', ec='green')
    
    # Draw arrow for second neighbor
    plt.arrow(x, y, np.cos(angle_rad) * magnitude, np.sin(angle_rad) * magnitude, head_width=0.15, head_length=0.15, fc='green', ec='green')
    
    # Draw a dot at moving points
    #for i in range(len(mpoints)):
    #    plt.scatter(mpoints[i][2], mpoints[i][1], color='green')
    
    # Plot the coordinate text
    for j in range(jdet_field.shape[0]):
        for i in range(jdet_field.shape[1]):
            # Get the pixel information
            normalized_y = j - (jdet_field.shape[0] // 2) 
            normalized_x = i - (jdet_field.shape[1] // 2)
            
            curr_displacement = deformation[:, 0, j, i]
            displacement_y = curr_displacement[1]
            displacement_x = curr_displacement[2]

            # Get the angle information
            curr_theta_rad = np.arctan2(displacement_y, displacement_x)
            if curr_theta_rad < 0:  # Ensure the angle is in the range [0, 2pi)
                curr_theta_rad += 2 * np.pi
            curr_theta = np.abs(np.degrees(curr_theta_rad))
            
            # Get the magnitude information
            curr_magnitude = np.linalg.norm(curr_displacement)
            jdet = jdet_field[j, i]
            
            # Setup text to display
            coord_text = f"({displacement_y:.1f}, {displacement_x:.1f})\n"
            jdet_text = f"{jdet:.2f} J"
            vector_text = f"\n{curr_theta:.2f}°\n" + f"∥{curr_magnitude:.2f}∥"
            #pixel_text = coord_text + jdet_text + vector_text
            pixel_text = jdet_text
            plt.text(i, j, pixel_text, color='black', ha='center', va='center', fontsize=fontsize, weight='normal')
            
            # Draw the vector direction
            magnitude_scale = 0.33  # To scale the vector magnitude for visualization purposes
            #plt.arrow(i, j, np.cos(curr_theta_rad) * magnitude_scale, np.sin(curr_theta_rad) * magnitude_scale, head_width=0.15, head_length=0.15, fc='black', ec='black')            

            
    

    # Set the xticks and yticks
    plt.xticks(np.arange(jdet_field.shape[0]), labels=np.arange(-(jdet_field.shape[1] // 2) - 1, jdet_field.shape[1] // 2) + 1)
    plt.yticks(np.arange(jdet_field.shape[0]), labels=np.arange(-(jdet_field.shape[0] // 2) - 1, jdet_field.shape[0] // 2) + 1)
    plt.title(title)
    plt.show()
    f.clear()
    plt.close(f)

In [12]:
def nested_dict():
    return defaultdict(nested_dict)

## Generate LUT data

Make original 7x7

In [ ]:
# User parameters
n = 7
SAVE_TO_FILE = True
pickle_filepath = 'output/final_case/data/lut_data_7x7.pkl'
multiplier = 1
USE_X_1_CASE = False

# Define a function to create a nested defaultdict
def nested_dict():
    return defaultdict(nested_dict)
data = nested_dict()

# Create a range of correspondences for each pixel
for y in tqdm(range(n), desc="Outer loop y"):
    for x in tqdm(range(n), desc="Outer loop x", leave=False):
        # Skip the center pixel
        if y == (n // 2) and x == (n // 2):
            continue
        new_y = y * multiplier
        new_x = x * multiplier
        # Create correspondences for each angle (degrees) [0, 360) in increments of 10]
        for angle in tqdm(range(0, 360, 10), desc="Inner loop angle", leave=False):
            
            # Create correspondences for each magnitude from [0.5, 5.0] in increments of 0.5
            for magnitude in tqdm(range(1, 11), desc="Innermost loop magnitude", leave=False):
                magnitude /= 2
                new_magnitude = magnitude * multiplier
                # Create the correspondence and apply as the second correspondence in the grid
                try:
                    deformation, jdet_field = create_case2(new_y, new_x, angle, new_magnitude, None, None, n=n*multiplier, base_magnitude=multiplier, x_1_case=USE_X_1_CASE)
                except Exception as e:
                    #print(f"Error: {e}")
                    #print(f"y: {y}, x: {x}, angle: {angle}, magnitude: {magnitude}")
                    continue
                data[y][x][angle][magnitude] = deformation, jdet_field


# Pickle the nested dictionary to a file
if SAVE_TO_FILE:
    with open(pickle_filepath, 'wb') as file:
        pickle.dump(data, file)

In [ ]:
csv_filepath = "output/final_case/lut_data_7x7.csv"

# Load the pickled dictionary from the file
with open('output/final_case/data/lut_data_7x7.pkl', 'rb') as file:
    data = pickle.load(file)
    
headers = ["x", "y", "theta", "mag", "J", "min_J"]
with open(csv_filepath, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    for x in tqdm(range(n), desc="Outer loop x"):
        
        for y in tqdm(range(n), desc="Outer loop y", leave=False):
            if y == (n // 2) and x == (n // 2):
                continue

            for angle in tqdm(range(0, 360, 10), desc="Inner loop angle", leave=False):
                for magnitude in tqdm(range(1, 11), desc="Innermost loop magnitude", leave=False):
                    magnitude /= 2
                    
                    try:
                        deformation_field, jdet_field = data[y][x][angle][magnitude]
                        jdet = jdet_field[n // 2, n // 2]
                        min_jdet = jdet_field.min()
                        normalized_y = y - (7 // 2)
                        normalized_x = x - (7 // 2)
                        writer.writerow([normalized_x, normalized_y, angle, magnitude, jdet, min_jdet])
                    except Exception as e:
                        continue

## Make 3x (21x21)

In [ ]:
# User parameters
n = 7
SAVE_TO_FILE = True
pickle_filepath = 'output/final_case/data/lut_data_21x21.pkl'
multiplier = 3
USE_X_1_CASE = False

# Define a function to create a nested defaultdict
def nested_dict():
    return defaultdict(nested_dict)
data = nested_dict()

# Create a range of correspondences for each pixel
for y in tqdm(range(n), desc="Outer loop y"):
    for x in tqdm(range(n), desc="Outer loop x", leave=False):
        # Skip the center pixel
        if y == (n // 2) and x == (n // 2):
            continue
        new_y = y * multiplier
        new_x = x * multiplier
        # Create correspondences for each angle (degrees) [0, 360) in increments of 10]
        for angle in tqdm(range(0, 360, 10), desc="Inner loop angle", leave=False):
            
            # Create correspondences for each magnitude from [0.5, 5.0] in increments of 0.5
            for magnitude in tqdm(range(1, 11), desc="Innermost loop magnitude", leave=False):
                magnitude /= 2
                new_magnitude = magnitude * multiplier
                # Create the correspondence and apply as the second correspondence in the grid
                try:
                    deformation, jdet_field = create_case2(new_y, new_x, angle, new_magnitude, None, None, n=n*multiplier, base_magnitude=multiplier, x_1_case=USE_X_1_CASE, downsample=True, multiplier=multiplier, upscaled_offset=1)
                except Exception as e:
                    #print(f"Error: {e}")
                    #print(f"y: {y}, x: {x}, angle: {angle}, magnitude: {magnitude}")
                    continue
                data[y][x][angle][magnitude] = deformation, jdet_field


# Pickle the nested dictionary to a file
if SAVE_TO_FILE:
    with open(pickle_filepath, 'wb') as file:
        pickle.dump(data, file)

In [ ]:
csv_filepath = "output/final_case/lut_data_21x21.csv"

# Load the pickled dictionary from the file
with open('output/final_case/data/lut_data_21x21.pkl', 'rb') as file:
    data = pickle.load(file)
    
headers = ["x", "y", "theta", "mag", "J", "min_J"]
with open(csv_filepath, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    for x in tqdm(range(n), desc="Outer loop x"):
        
        for y in tqdm(range(n), desc="Outer loop y", leave=False):
            if y == (n // 2) and x == (n // 2):
                continue

            for angle in tqdm(range(0, 360, 10), desc="Inner loop angle", leave=False):
                for magnitude in tqdm(range(1, 11), desc="Innermost loop magnitude", leave=False):
                    magnitude /= 2
                    
                    try:
                        deformation_field, jdet_field = data[y][x][angle][magnitude]
                        jdet = jdet_field[n // 2, n // 2]
                        min_jdet = jdet_field.min()
                        normalized_y = y - (7 // 2)
                        normalized_x = x - (7 // 2)
                        writer.writerow([normalized_x, normalized_y, angle, magnitude, jdet, min_jdet])
                    except Exception as e:
                        continue

Make 9x (63x63)

In [ ]:
# User parameters
n = 7
SAVE_TO_FILE = True
pickle_filepath = 'output/final_case/data/lut_data_63x63.pkl'
multiplier = 9
USE_X_1_CASE = False

# Define a function to create a nested defaultdict
def nested_dict():
    return defaultdict(nested_dict)
data = nested_dict()

# Create a range of correspondences for each pixel
for y in tqdm(range(n), desc="Outer loop y"):
    for x in tqdm(range(n), desc="Outer loop x", leave=False):
        # Skip the center pixel
        if y == (n // 2) and x == (n // 2):
            continue
        new_y = y * multiplier
        new_x = x * multiplier
        # Create correspondences for each angle (degrees) [0, 360) in increments of 10]
        for angle in tqdm(range(0, 360, 10), desc="Inner loop angle", leave=False):
            
            # Create correspondences for each magnitude from [0.5, 5.0] in increments of 0.5
            for magnitude in tqdm(range(1, 11), desc="Innermost loop magnitude", leave=False):
                magnitude /= 2
                new_magnitude = magnitude * multiplier
                # Create the correspondence and apply as the second correspondence in the grid
                try:
                    deformation, jdet_field = create_case2(new_y, new_x, angle, new_magnitude, None, None, n=n*multiplier, base_magnitude=multiplier, x_1_case=USE_X_1_CASE, downsample=True, multiplier=multiplier, upscaled_offset=4)
                except Exception as e:
                    #print(f"Error: {e}")
                    #print(f"y: {y}, x: {x}, angle: {angle}, magnitude: {magnitude}")
                    continue
                data[y][x][angle][magnitude] = deformation, jdet_field


# Pickle the nested dictionary to a file
if SAVE_TO_FILE:
    with open(pickle_filepath, 'wb') as file:
        pickle.dump(data, file)

In [ ]:
# Load the pickled dictionary from the file
with open('output/final_case/data/lut_data_63x63.pkl', 'rb') as file:
    data = pickle.load(file)
    
csv_filepath = "output/final_case/lut_data_63x63.csv"

headers = ["x", "y", "theta", "mag", "J", "min_J"]
with open(csv_filepath, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    for x in tqdm(range(n), desc="Outer loop x"):
        
        for y in tqdm(range(n), desc="Outer loop y", leave=False):
            if y == (n // 2) and x == (n // 2):
                continue

            for angle in tqdm(range(0, 360, 10), desc="Inner loop angle", leave=False):
                for magnitude in tqdm(range(1, 11), desc="Innermost loop magnitude", leave=False):
                    magnitude /= 2
                    
                    try:
                        deformation_field, jdet_field = data[y][x][angle][magnitude]
                        jdet = jdet_field[n // 2, n // 2]
                        min_jdet = jdet_field.min()
                        normalized_y = y - (7 // 2)
                        normalized_x = x - (7 // 2)
                        writer.writerow([normalized_x, normalized_y, angle, magnitude, jdet, min_jdet])
                    except Exception as e:
                        continue

## Save to CSV spreadsheet

In [8]:
new_y = 0
new_x = 0
angle = 0
new_magnitude = 1
multiplier = 3
n = 7

deformation, jdet_field = create_case2(new_y, new_x, angle, new_magnitude, 
                                       None, None, n=n*multiplier, base_magnitude=multiplier, 
                                       x_1_case=False, downsample=True, multiplier=multiplier, upscaled_offset=1)

In [ ]:
jdet_field

In [ ]:
from skimage.measure import block_reduce
import numpy as np
import csv
from tqdm import tqdm
import pickle

n = 7

# Load the pickled dictionary from the file
with open('output/final_case/data/lut_data_7x7.pkl', 'rb') as file:
    data = pickle.load(file)
    
with open('output/final_case/data/lut_data_21x21.pkl', 'rb') as file:
    data3 = pickle.load(file)
    
with open('output/final_case/data/lut_data_63x63.pkl', 'rb') as file:
    data9 = pickle.load(file)

    
csv_filepath = 'output/final_case/lut_data_final.csv'

headers = ["x", "y", "theta", "mag", "J", "min_J", "J3", "min_J3", "J9", "min_J9", "correlation_J_J3", "correlation_J_J9", "correlation_J3_J9"]
with open(csv_filepath, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    for x in range(n):
        
        for y in range(n):
            if y == (n // 2) and x == (n // 2):
                continue

            for angle in range(0, 360, 10):
                for magnitude in range(1, 11):
                    magnitude /= 2
                    
                    try:
                        # Original data
                        deformation_field, jdet_field = data[y][x][angle][magnitude]
                        jdet = jdet_field[n // 2, n // 2]
                        min_jdet = jdet_field.min()
                        normalized_y = y - (jdet_field.shape[0] // 2)
                        normalized_x = x - (jdet_field.shape[1] // 2)
                        
                        # Scaled data
                        deformation_field3, jdet_field3 = data3[y][x][angle][magnitude]
                        jdet3 = jdet_field3[n // 2, n // 2]
                        min_jdet3 = jdet_field3.min()
                        
                        # Scaled data
                        deformation_field9, jdet_field9 = data9[y][x][angle][magnitude]
                        jdet9 = jdet_field9[n // 2, n // 2]
                        min_jdet9 = jdet_field9.min()
                        
                        # Calculate correlation
                        #jdet_field3_downsampled = block_reduce(jdet_field3, (3, 3), np.mean)
                        #jdet_field9_downsampled = block_reduce(jdet_field9, (9, 9), np.mean)
                        
                        correlation = np.corrcoef(jdet_field.flatten(), jdet_field3.flatten())[0, 1]
                        correlation2 = np.corrcoef(jdet_field.flatten(), jdet_field9.flatten())[0, 1]
                        correlation3 = np.corrcoef(jdet_field3.flatten(), jdet_field9.flatten())[0, 1]
                        
                        writer.writerow([normalized_x, normalized_y, angle, magnitude, jdet, min_jdet, jdet3, min_jdet3, jdet9, min_jdet9, correlation, correlation2, correlation3])
                    except Exception as e:
                        continue

## Load data and visualize it

In [60]:
# Load the pickled dictionary from the file
with open('lut_data_case2_x-1.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

In [ ]:
# Show grid information at a specific index
y, x = (-2, 2)
angle = 180
magnitude = 4
n = 7

y = y + n // 2
x = x + n // 2
# Get info
deformation, jdet_field = loaded_data[y][x][angle][magnitude]
central_jdet = jdet_field[n//2, n//2]

deform = deformation[:, 0, :, :]
pprint(deform)
#deformation, _ = create_case(y, x, angle, magnitude, mpoints, fpoints, n)

print("Central Jacobian determinant:", central_jdet)
show_data(loaded_data, y, x, angle, magnitude, 
          title=f"Correspondence at ({y - n // 2}, {x - n // 2})\ntheta={angle}°, mag={magnitude}", fontsize=6)

In [ ]:
# Show grid information at a specific index
y, x = (-2, 2)
angle = 180
magnitude = 4
n = 7

y = y + n // 2
x = x + n // 2
# Get info
deformation, jdet_field = loaded_data[y][x][angle][magnitude]
central_jdet = jdet_field[n//2, n//2]

deform = deformation[:, 0, :, :]
pprint(deform)
#deformation, _ = create_case(y, x, angle, magnitude, mpoints, fpoints, n)

print("Central Jacobian determinant:", central_jdet)
show_data(loaded_data, y, x, angle, magnitude, 
          title=f"Correspondence at ({y - n // 2}, {x - n // 2})\ntheta={angle}°, mag={magnitude}", fontsize=6)